# Receipt printing in .NET

Reference the `System.Drawing.Common` package:

In [208]:
#r "nuget: System.Drawing.Common"
using System.Drawing;
using System.Drawing.Printing;

Installed Packages System.Drawing.Common, 9.0.9

List all available printers:

In [209]:
foreach (var printerName in PrinterSettings.InstalledPrinters) {
    Console.WriteLine(printerName);
}

POSPrinter POS58
OneNote (Desktop)
Microsoft Print to PDF
Canon MF645C
Adobe PDF


Create the `PrintDocument` object for a given printer:

In [210]:
var doc = new PrintDocument() {
    PrinterSettings = new PrinterSettings {
        PrinterName = "POSPrinter POS58"
    }
};

List all available paper sizes:

In [211]:
var paperSizes = doc.PrinterSettings.PaperSizes.Cast<PaperSize>();
paperSizes.DisplayTable();

Height,Kind,PaperName,RawKind,Width
827,Custom,58(48)mm x 210 mm,257,189
1169,Custom,58(48)mm x 297mm,258,189
-12821,Custom,58(48)mm x 3297mm,259,189
827,Custom,Custom size,256,189


Set the paper size:

In [212]:
doc.DefaultPageSettings.PaperSize = paperSizes.First(ps => ps.PaperName == "58(48)mm x 3297mm");

In [ ]:
doc.PrintPage += (s, e) => {
    var y = 0;
    var total = 0f;
    var g = e.Graphics;

    void AddText(string text, Font font, StringAlignment alignment = StringAlignment.Near, bool lineFeed = true) {
        // Define string format with specified alignment
        var textFormat = new StringFormat {
            Alignment = alignment,
        };

        // Define layout rectangle
        var textRect = new Rectangle(0, y, e.PageBounds.Width, 0);

        // Draw the string within the layout rectangle
        g.DrawString(text, font, Brushes.Black, textRect, textFormat);

        // Measure the size of the drawn text to update y position
        if(lineFeed) {
            var textSize = g.MeasureString(text, font, textRect.Size);
            y += (int)textSize.Height;
        }
    }

    void AddImage(Image img) {
        // Calculate image rectangle to fit the page width while maintaining aspect ratio
        var imgRect = new Rectangle(
            0, y,                                                       // Position (top left)
            e.PageBounds.Width,                                         // Width - fit to page width
            (int)((float)e.PageBounds.Width / img.Width * img.Height)); // Height - keep aspect ratio

        // Draw the image
        g.DrawImage(img, imgRect);

        // Update y position
        y += imgRect.Bottom;
    }

    void AddItem(string text, float unitPrice, int quantity, Font font) {
        AddText(text, font);
        if(quantity == 1) {
            AddText($"{unitPrice:0.00} Kč", font, StringAlignment.Far);
        } else {
            AddText($"{quantity} x {unitPrice:0.00} = {unitPrice * quantity:0.00} Kč", font, StringAlignment.Far);
        }
        total += unitPrice * quantity;
    }

    void AddLine(int height) {
        using var pen = new Pen(Color.Black, height);
        g.DrawLine(pen, 0, y, e.PageBounds.Width, y);
        y += height;
    }

    // Header image
    var img = Image.FromFile(@"D:\Documents\Grafika\Altairis\Logo\standard\altairis_logo_bw_standard.png");
    AddImage(img);
    y += 5;

    // Header text
    using var defaultFont = new Font("Arial", 8);
    AddText("Bořivojova 35, 130 00 Praha 3", defaultFont, StringAlignment.Center);
    AddText("IČO: 27560911 | DIČ: CZ27560911", defaultFont, StringAlignment.Center);
    y+= 10;

    // Some sample items
    AddItem("Položka 1", 49.00f, 1, defaultFont);
    AddItem("Nějaký opravdu velmi dlouhý název položky 2", 199.00f, 2, defaultFont);
    AddItem("Položka 3", 9.90f, 1, defaultFont);

    // Total
    using var boldFont = new Font("Arial", 8, FontStyle.Bold);
    AddText("Celkem", boldFont, StringAlignment.Near, lineFeed:false);
    AddText($"{total:0.00} Kč", boldFont, StringAlignment.Far);
};
doc.Print();